# Preparation

In [0]:
# Install pip libraries
!pip3 install lxml
!pip3 install html5lib

In [0]:
# Import libraries
import numpy as np
import pandas as pd
import pyspark as ps
import pyspark.pandas as pspa
from pyspark.sql import functions as F

# 1. Extraction

## 1.1. Import tables from URLs (as lists of pandas dataframes)

In [0]:
# Define source URLs
url_top_channels = 'http://us.youtubers.me/global/all/top-1000-youtube-channels'
url_top_videos = 'https://us.youtubers.me/global/all/top-1000-youtube-videos'
url_most_subscribed_channels = 'https://us.youtubers.me/global/all/top-1000-most-subscribed-youtube-channels'
url_most_popular_channels = 'https://us.youtubers.me/global/all/top-1000-most-popular-youtube-channels'
url_most_popular_videos = 'https://us.youtubers.me/global/all/top-1000-most-popular-youtube-videos'

# Import df lists from source URLs
dflist_top_channels = pd.read_html(io = url_top_channels)
dflist_top_videos = pd.read_html(io = url_top_videos)
dflist_most_subscribed_channels = pd.read_html(io = url_top_videos)
dflist_most_popular_channels = pd.read_html(io=url_most_popular_channels)
dflist_most_popular_videos = pd.read_html(io=url_most_popular_videos)

## 1.2. Extract dataframes from lists

Each dataframe list has 3 elements:
- Element 1: Dataframe 1
- Element 2: Dataframe 2 (part 1)
- Element 3: Dataframe 2 (part 2)

In order to create DF2, we must .concat() E2 and E3.

In [0]:
# Create dataframes from E1s
df_top_channels = dflist_top_channels[0]
df_top_videos = dflist_top_videos[0]
df_most_subscribed_channels = dflist_most_subscribed_channels[0]
df_most_popular_channels = dflist_most_popular_channels[0]
df_most_popular_videos = dflist_most_popular_videos[0]

# Rename E3s' column names to match E2s'
dflist_top_channels[2].columns = dflist_top_channels[1].columns.values.tolist()
dflist_top_videos[2].columns = dflist_top_videos[1].columns.values.tolist()
dflist_most_subscribed_channels[2].columns = dflist_most_subscribed_channels[1].columns.values.tolist()
dflist_most_popular_channels[2].columns = dflist_most_popular_channels[1].columns.values.tolist()
dflist_most_popular_videos[2].columns = dflist_most_popular_videos[1].columns.values.tolist()

# Create dataframes by concating E2s and E3s
df_top_channels_metrics = pd.concat(dflist_top_channels[1:])
df_top_videos_metrics = pd.concat(dflist_top_videos[1:])
df_most_subscribed_channels_metrics = pd.concat(dflist_most_subscribed_channels[1:])
df_most_popular_channels_metrics = pd.concat(dflist_most_popular_channels[1:])
df_most_popular_videos_metrics = pd.concat(dflist_most_popular_videos[1:])

## 1.3. Convert dataframes from pandas to spark

In [0]:
# Convert created pandas dataframes to spark dataframes
df_top_channels = spark.createDataFrame(df_top_channels)
df_top_channels_metrics = spark.createDataFrame(df_top_channels_metrics)

df_top_videos = spark.createDataFrame(df_top_videos)
df_top_videos_metrics = spark.createDataFrame(df_top_videos_metrics)

df_most_subscribed_channels = spark.createDataFrame(df_most_subscribed_channels)
df_most_subscribed_channels_metrics = spark.createDataFrame(df_most_subscribed_channels_metrics)

df_most_popular_channels = spark.createDataFrame(df_most_popular_channels)
df_most_popular_channels_metrics = spark.createDataFrame(df_most_popular_channels_metrics)

df_most_popular_videos = spark.createDataFrame(df_most_popular_videos)
df_most_popular_videos_metrics = spark.createDataFrame(df_most_popular_videos_metrics)

## 1.4. Display imported, raw dataframes

In [0]:
# Display all imported, raw dataframes
print('\nTop Channels')
display(df_top_channels)

print('\nTop Channels (metrics)')
display(df_top_channels_metrics)

print('\nTop Videos')
display(df_top_videos)

print('\nTop Videos (metrics)')
display(df_top_videos_metrics)

print('\nMost Subscribed Channels')
display(df_most_subscribed_channels)

print('\nMost Subscribed Channels (metrics)')
display(df_most_subscribed_channels_metrics)

print('\nMost Popular Channels')
display(df_most_popular_channels)

print('\nMost Popular Channels (metrics)')
display(df_most_popular_channels_metrics)

print('\nMost Popular Videos')
display(df_most_popular_videos)

print('\nMost Popular Videos (metrics)')
display(df_most_popular_videos_metrics)

## 1.5. Save raw dataframes as a parquet files (1st layer)

In [0]:
# Delete raw data from previous runs
dbutils.fs.rm('dbfs:/FileStore/raw', True)

# Save dataframes as parquet files
df_top_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/topChannels')

df_top_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/topChannelsMetrics')

df_top_videos\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/topVideos')

df_top_videos_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/topVideosMetrics')

df_most_subscribed_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostSubscribed')

df_most_subscribed_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostSubscribedMetrics')

df_most_popular_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostPopularChannels')

df_most_popular_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostPopularChannelsMetrics')

df_most_popular_videos\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostPopularVideos')

df_most_popular_videos_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostPopularVideosMetrics')

# 2. Transformation

## 2.1. Top Channels

### Clean column headers

In [0]:
# Show dataframe column names and types
df_top_channels.printSchema()

# Create list with stripped, lowercased, snake-cased column headers
newColumns = [column.strip()\
                    .lower()\
                    .replace(' ', '_')
              for column
              in df_top_channels.columns]

# Create dataframe with new column headers
df_top_channels = df_top_channels.toDF(*newColumns)

# Show dataframe with updated column names
df_top_channels.printSchema()

### Clean column values

In [0]:
# Show dataframe values
display(df_top_channels)

# Create dataframe with clean values (trim left/right whitespace, lowercase STRs, remove commas from numbers)
df_top_channels = df_top_channels\
    .withColumn('youtuber', F.trim(df_top_channels['youtuber']))\
    .withColumn('youtuber', F.lower(F.col('youtuber')))\
    .withColumn('subscribers', F.translate('subscribers', ',', ''))\
    .withColumn('video_views', F.translate('video_views', ',', ''))\
    .withColumn('video_count', F.translate('video_count', ',', ''))\
    .withColumn('category', F.trim(df_top_channels['category']))\
    .withColumn('category', F.lower(F.col('category')))

# Show dataframe with updated values
display(df_top_channels)

### Update data types

In [0]:
# Show dataframe column names and types
df_top_channels.printSchema()

In [0]:
# Show dataframe column names and types
df_top_channels.printSchema()

# Create dataframe with updated data types
df_top_channels = df_top_channels\
    .withColumn('rank', F.col('rank').cast('int'))\
    .withColumn('subscribers', F.col('subscribers').cast('int'))\
    .withColumn('video_views', F.col('video_views').cast('long'))\
    .withColumn('video_count', F.col('video_count').cast('int'))\
    .withColumn('started', F.col('started').cast('int'))

# Show dataframe with updated data types
df_top_channels.printSchema()

## 2.2. Top Videos

### Clean column headers

In [0]:
# Show dataframe column names and types
df_top_videos.printSchema()

# Create list with stripped, lowercased, snake-cased column headers
newColumns = [column.strip()\
                    .lower()\
                    .replace(' ', '_')
              for column
              in df_top_videos.columns]

# Create dataframe with new column headers
df_top_videos = df_top_videos.toDF(*newColumns)

# Show dataframe with updated column names
df_top_videos.printSchema()

### Clean column values

In [0]:
# Show dataframe values
display(df_top_videos)

# Create dataframe with clean values (trim left/right whitespace, lowercase STRs, remove commas from numbers)
df_top_videos = df_top_videos\
    .withColumn('video', F.trim(df_top_videos['video']))\
    .withColumn('video', F.lower(F.col('video')))\
    .withColumn('category', F.trim(df_top_videos['category']))\
    .withColumn('category', F.lower(F.col('category')))

# Show dataframe with updated values
display(df_top_videos)

### Update data types

In [0]:
# Show dataframe column names and types
df_top_videos.printSchema()

In [0]:
# Show dataframe column names and types
df_top_videos.printSchema()

# Create dataframe with updated data types
df_top_videos = df_top_videos\
    .withColumn('video_views', F.col('video_views').cast('long'))\
    .withColumn('likes', F.col('likes').cast('long'))\
    .withColumn('published', F.col('published').cast('int'))

# Show dataframe with updated data types
df_top_videos.printSchema()

## 2.3. Top Channels (metrics)

## 2.4. Top Videos (metrics)

## 2.5. Most Subscribed Channels

### clean collums header

In [0]:
# Show dataframe column names and types
df_most_subscribed_channels.printSchema()

# Create list with stripped, lowercased, snake-cased column headers
newColumns = [column.strip()\
                    .lower()\
                    .replace(' ', '_')
              for column
              in df_most_subscribed_channels.columns]

# Create dataframe with new column headers
df_most_subscribed_channels = df_most_subscribed_channels.toDF(*newColumns)

# Show dataframe with updated column names
df_most_subscribed_channels.printSchema()

### clean collums values

In [0]:
# Show dataframe values
display(df_most_subscribed_channels)

# Create dataframe with clean values (trim left/right whitespace, lowercase STRs, remove commas from numbers)
df_most_subscribed_channels = df_most_subscribed_channels\
    .withColumn('youtuber', F.trim(df_most_subscribed_channels['youtuber']))\
    .withColumn('youtuber', F.lower(F.col('youtuber')))\
    .withColumn('category', F.trim(df_most_subscribed_channels['category']))\
    .withColumn('category', F.lower(F.col('category')))

# Show dataframe with updated values
display(df_most_subscribed_channels)

## 2.6. Most Subscribed Channels (metrics)

### clean collums header

In [0]:
# Show dataframe column names and types
df_most_subscribed_channels_metrics.printSchema()

# Create list with stripped, lowercased, snake-cased column headers
newColumns = [column.strip()\
                    .lower()\
                    .replace(' ', '_')
              for column
              in df_most_subscribed_channels_metrics.columns]

# Create dataframe with new column headers
df_most_subscribed_channels_metrics = df_most_subscribed_channels_metrics.toDF(*newColumns)

# Show dataframe with updated column names
df_most_subscribed_channels_metrics.printSchema()

### clean collums values

In [0]:
# Show dataframe values
display(df_most_subscribed_channels_metrics)

# Create dataframe with clean values (trim left/right whitespace, lowercase STRs, remove commas from numbers)
df_most_subscribed_channels_metrics = df_most_subscribed_channels_metrics\
    .withColumn('video', F.trim(df_most_subscribed_channels_metrics['video']))\
    .withColumn('video', F.lower(F.col('video')))

# Show dataframe with updated values
display(df_most_subscribed_channels_metrics)

## 2.7. Most Popular Channels

### clean collums header

In [0]:
# Show dataframe column names and types
df_most_popular_channels.printSchema()

# Create list with stripped, lowercased, snake-cased column headers
newColumns = [column.strip()\
                    .lower()\
                    .replace(' ', '_')
              for column
              in df_most_popular_channels.columns]

# Create dataframe with new column headers
df_most_popular_channels = df_most_popular_channels.toDF(*newColumns)

# Show dataframe with updated column names
df_most_popular_channels.printSchema()

### clean collums values

In [0]:
# Show dataframe values
display(df_most_popular_channels)

# Create dataframe with clean values (trim left/right whitespace, lowercase STRs, remove commas from numbers)
df_most_popular_channels = df_most_popular_channels\
    .withColumn('youtuber', F.trim(df_most_subscribed_channels['youtuber']))\
    .withColumn('youtuber', F.lower(F.col('youtuber')))\
    .withColumn('category', F.trim(df_most_subscribed_channels['category']))\
    .withColumn('category', F.lower(F.col('category')))

# Show dataframe with updated values
display(df_most_popular_channels)

## 2.8. Most Popular Channels (metrics)

### clean collums header

In [0]:
# Show dataframe column names and types
df_most_popular_channels_metrics.printSchema()

# Create list with stripped, lowercased, snake-cased column headers
newColumns = [column.strip()\
                    .lower()\
                    .replace(' ', '_')
              for column
              in df_most_popular_channels_metrics.columns]

# Create dataframe with new column headers
df_most_popular_channels_metrics = df_most_popular_channels_metrics.toDF(*newColumns)

# Show dataframe with updated column names
df_most_popular_channels_metrics.printSchema()

### clean collums values

In [0]:
# Show dataframe values
display(df_most_popular_channels_metrics)

# Create dataframe with clean values (trim left/right whitespace, lowercase STRs, remove commas from numbers)
df_most_popular_channels_metrics = df_most_popular_channels_metrics\
    .withColumn('youtuber', F.trim(df_most_subscribed_channels_metrics['youtuber']))\
    .withColumn('youtuber', F.lower(F.col('youtuber')))

# Show dataframe with updated values
display(df_most_popular_channels_metrics)

## 2.9. Most Popular Videos

### clean collums header

In [0]:
# Show dataframe column names and types
df_most_popular_videos.printSchema()

# Create list with stripped, lowercased, snake-cased column headers
newColumns = [column.strip()\
                    .lower()\
                    .replace(' ', '_')
              for column
              in df_most_popular_videos.columns]

# Create dataframe with new column headers
df_most_popular_videos = df_most_popular_videos.toDF(*newColumns)

# Show dataframe with updated column names
df_most_popular_videos.printSchema()

### clean collums values

In [0]:
# Show dataframe values
display(df_most_popular_videos)

# Create dataframe with clean values (trim left/right whitespace, lowercase STRs, remove commas from numbers)
df_most_popular_videos = df_most_popular_videos\
    .withColumn('video', F.trim(df_most_popular_videos['video']))\
    .withColumn('video', F.lower(F.col('video')))\
    .withColumn('category', F.trim(df_most_popular_videos['category']))\
    .withColumn('category', F.lower(F.col('category')))

# Show dataframe with updated values
display(df_most_popular_videos)

## 2.10. Most Popular Videos (metrics)

### clean collums header

In [0]:
# Show dataframe column names and types
df_most_popular_videos_metrics.printSchema()

# Create list with stripped, lowercased, snake-cased column headers
newColumns = [column.strip()\
                    .lower()\
                    .replace(' ', '_')
              for column
              in df_most_popular_videos_metrics.columns]

# Create dataframe with new column headers
df_most_popular_videos_metrics = df_most_popular_videos_metrics.toDF(*newColumns)

# Show dataframe with updated column names
df_most_popular_videos_metrics.printSchema()

### clean collums values

In [0]:
# Show dataframe values
display(df_most_popular_videos_metrics)

# Create dataframe with clean values (trim left/right whitespace, lowercase STRs, remove commas from numbers)
df_most_popular_videos_metrics = df_most_popular_videos_metrics\
    .withColumn('video', F.trim(df_most_popular_videos_metrics['video']))\
    .withColumn('video', F.lower(F.col('video')))

# Show dataframe with updated values
display(df_most_popular_videos_metrics)

## 2.11. Save refined dataframes as parquet files (2nd layer)

In [0]:
# Delete refined data from previous runs
dbutils.fs.rm('dbfs:/FileStore/refined', True)

# Save dataframes as parquet files
df_top_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/topChannels')

df_top_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/topChannelsMetrics')

df_top_videos\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/topVideos')

df_top_videos_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/topVideosMetrics')

df_most_subscribed_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostSubscribed')

df_most_subscribed_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostSubscribedMetrics')

df_most_popular_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostPopularChannels')

df_most_popular_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostPopularChannelsMetrics')

df_most_popular_videos\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostPopularVideos')

df_most_popular_videos_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostPopularVideosMetrics')